In [1]:
# The ultimate target feature: time from one stop to another

# Imports
import pandas as pd
import numpy as np
from datetime import date, datetime
from patsy import dmatrices
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import export_graphviz, DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.tree import export_graphviz 
from sklearn import tree
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
from statsmodels.formula.api import ols
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Read csv file into a dataframe.
df = pd.read_csv('csv_data/beta4.csv')
df.head()

/Users/katerooney/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,Unnamed: 0,Timestamp,LineID,Direction,Journey_Pattern_ID,Timeframe,Vehicle_Journey_ID,Lon,Lat,Vehicle_ID,...,End_Stop,Stops_To_Travel,Time_To_Travel_Dirty,Time_To_Travel,Scheduled_Speed_Per_Stop,time_bins,Rain,Temperature,Wind_Speed,Hour_Of_Day_y
0,143,1352181949000000,4,0,40001,2012-11-06,5332,-6.264970,53.402534,43034,...,57,52,2637000000,43.0,1.016949,4,0.0,2.2,10.0,23
1,177,1352182129000000,4,0,40001,2012-11-06,5332,-6.264059,53.396149,43034,...,57,52,2457000000,40.0,1.016949,4,0.0,2.2,10.0,23
2,196,1352182208000000,4,0,40001,2012-11-06,5332,-6.263807,53.392265,43034,...,57,52,2378000000,39.0,1.016949,4,0.0,2.2,10.0,23
3,229,1352182391000000,4,0,40001,2012-11-06,5332,-6.265748,53.380424,43034,...,57,52,2195000000,36.0,1.016949,4,0.0,2.2,10.0,23
4,247,1352182484000000,4,0,40001,2012-11-06,5317,-6.225992,53.327579,43032,...,59,19,802000000,13.0,1.016949,1,0.0,2.2,10.0,23


# 1.0 Model Analysis

## 1.1 XG Boost

In [2]:
X = df[["Day_Of_Week" , "Time_Bin_Start" , "Scheduled_Speed_Per_Stop", "Wind_Speed", "Temperature", "Holiday", "Stops_To_Travel", "Stop_Sequence"]]
Y = df['Time_To_Travel']

In [9]:
encoded_x = None
for i in range(0, X.shape[1]):
	label_encoder = LabelEncoder()
	feature = label_encoder.fit_transform(X[["Day_Of_Week"]])
	feature = feature.reshape(X.shape[0], 1)
	onehot_encoder = OneHotEncoder(sparse=False)
	feature = onehot_encoder.fit_transform(feature)
	if encoded_x is None:
		encoded_x = feature
	else:
		encoded_x = np.concatenate((encoded_x, feature), axis=1)
print("X shape: : ", encoded_x.shape)
# encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Y)
label_encoded_y = label_encoder.transform(Y)

X shape: :  (68575, 56)


/Users/katerooney/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
encoded_x = None
for i in range(0, X.shape[1]):
	label_encoder = LabelEncoder()
	feature = label_encoder.fit_transform(X[["Time_Bin_Start"]])
	feature = feature.reshape(X.shape[0], 1)
	onehot_encoder = OneHotEncoder(sparse=False)
	feature = onehot_encoder.fit_transform(feature)
	if encoded_x is None:
		encoded_x = feature
	else:
		encoded_x = np.concatenate((encoded_x, feature), axis=1)
print("X shape: : ", encoded_x.shape)
# encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Y)
label_encoded_y = label_encoder.transform(Y)

/Users/katerooney/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


X shape: :  (68575, 584)


In [11]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=33) 

In [12]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [13]:
y_pred = model.predict(X_test)

In [14]:
predictions = [round(value) for value in y_pred]

In [15]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 11.89%
